In [ ]:
__autor__ = "Felipe Alejandro Pizarro Márquez"
__credits__ = ["Felipe Alejandro Pizarro Márquez",]
__email__ = "fpizarro92@live.cl"
__status__ = "Development"

## Import image set and call external sofware for photogrammetry

This process take a set of images and create a fully-textured 3D mesh using photogrammetry. To accomplish this, first it generate a dense cloud point with the image set using [openMVG](https://github.com/openMVG/openMVG) . Then, it uses [openMVS](https://github.com/cdcseacave/openMVS) to generate a fully-textured 3D mesh. 

In [ ]:
import os
import subprocess
import shutil
import PIL.Image
import blendjupyter as blendj
import fwdimaging as fwd
from tkinter import *
from tkinter import filedialog

In [ ]:
def create_3dmodel(**kwargs):
    """     
    Optional Arguments:
        outdirname: The name of the project
        inpath:  It's the folder where the original images are stored.
        outpath: It's the absolute path of the output folder.
        scale: Scale the images, from 0 to 1
        reslevel: for openMVS
        gray: Transform input images to grayscale
    """ 
    reslevel = kwargs.pop('reslevel', 0)
    scale = kwargs.pop('scale', 1)
    gray = kwargs.pop('gray', False)
    sample = kwargs.pop('sample', False)     
    s3path = kwargs.pop('s3path',os.path.join(os.path.expanduser('~'),
                                              'super-scanner-software-s3'))   
    inpath = kwargs.pop('inpath',os.path.join(os.path.expanduser('~'),
                                              's3-out','scanner',
                                              'blend-phg-set-0001'))    
    outdirname = kwargs.pop('outdirname', 
                            os.path.join(inpath,'phg-model-0001'))  
    samplepath = os.path.join(s3path,"img","photog")
    
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(create_3dmodel.__name__, list(kwargs.keys())[-1]))
    
    # Sample project, if sample is true,   
    # the program uses artemis-statue as inpath
    if sample is True:
        inpath = os.path.join(samplepath,'artemis-statue')  
        outdirname = os.path.join(os.path.expanduser('~'),
                                  's3-out','scanner',
                                  'artemis-statue-example',
                                  'phg-model-0001')                
    outpath = kwargs.pop('outpath',
                         fwd.find_out_dir(dirname=outdirname,
                                          parentdir='scanner')) 
    imgpath = os.path.join(outpath,"res-images")
    print("Input image path: %s" %inpath)
    print("Output path: %s"  %outpath)     
    #copy and scale the images to the project path
    if scale >= 1:
        man_input_img(inpath,imgpath,gray = gray)
    else:
        man_input_img(inpath,imgpath,
                      gray = gray, scale = scale)    
    mvgmvspipeline = os.path.join(s3path,
                                  "scripts",
                                  "mvgmvspipeline.py")    
    #call the python script pipeline to use openmvg
    #and openmvs with custom parameters 
    subprocess.call(['python3',mvgmvspipeline,
                     imgpath,outpath,
                     "--1","m","AKAZE_FLOAT","p","HIGH",
                     "--2","r","0.7",
                     "--8","resolution-level",str(reslevel),
                     "--10","resolution-level",str(reslevel),
                     "--11","resolution-level",str(reslevel)])     
    return outpath   

In [ ]:
def get_3dmodel(**kwargs):
    """
    Optional Arguments:
        editor: It's the object's editor. Currently only MESHLAB.
        mesh = textured or model, default is textured
        projpath: It's project path.  
    """    
    if kwargs:
        raise TypeError('{!s}() got an unexpected keyword argument {!r}'.format(get_3dmodel.__name__, list(kwargs.keys())[-1]))
    #Check if the selected editor exist
    editor = kwargs.pop('editor','meshlab')
    editor_switcher = {"meshlab": "meshlab"}
    editor = editor_switcher.get(editor.lower(),False)       
    assert editor != False,"incorrect editor"
    #Check the mesh type
    mesh = kwargs.pop('mesh','textured')
    mesh_switcher = {"textured": "textured","model":"model"}
    mesh = mesh_switcher.get(mesh.lower(),False)       
    assert editor != False,"incorrect editor"    
    #The project path
    projpath = kwargs.pop('projpath', False)
    outpath = os.path.join(os.path.expanduser('~'),'s3-out')
    #Check if the project exist
    if (projpath is False):   
        mkdir_ine(outpath)
        projpath = select_folder(initialdir=outpath,
                                     title = "Select project folder.")
    assert len(projpath) is not 0,"no project folder selected"    
    print("Project path: %s"  % projpath)
    if mesh is "model":
        obj = os.path.join(projpath, "mvs",
                           "scene_dense_mesh_refine.ply")
    if mesh is "textured":
        obj = os.path.join(projpath, "mvs",
                           "scene_dense_mesh_refine_texture.ply")
    assert os.path.isfile(obj) is True, "incorrect project"     
    if editor is "meshlab":
    #Use meshlab to edit the mesh        
        subprocess.Popen([editor,obj])

In [ ]:
def mkdir_ine(dirname):
    """Create the folder if not presents"""
    if not os.path.exists(dirname):
        os.makedirs(dirname)

In [ ]:
def select_folder(**kwargs):
    """
    Return a folder path selected from tkinter askdirectory
    Optional Arguments:
        initial_dir:  It's where to start looking for a folder
        title:        The title of the windows screen
    """
    initialdir = kwargs.pop('initialdir', os.path.expanduser('~'))
    title = kwargs.pop('title',
                       'Please choose a directory, then press OK.')
    root = Tk()
    root.attributes("-topmost", True)
    root.withdraw()
    return filedialog.askdirectory(initialdir=initialdir,
                                   parent=root,title = title,
                                   mustexist = True)

In [ ]:
def man_input_img(input_path,output_path,**kwargs):
    """
    Manages the input image path
    """
    scale = kwargs.pop('scale', 1)
    gray = kwargs.pop('gray', False)
    exts = ('.jpg', '.JPG')
    os.makedirs(output_path) 
    for file in os.listdir(input_path):
        if os.path.isfile(os.path.join(input_path, file)):
            if any(file.lower().endswith(ext) for ext in exts):
                shutil.copy2(os.path.join(input_path, file),
                             os.path.join(output_path,file))
                if scale != 1 or gray == True:
                    resize_image(os.path.join(output_path,file),
                                 scale, gray = gray)
                    blendj.copy_exif(os.path.join(output_path,file),
                                     os.path.join(input_path, file))

In [ ]:
def resize_image(img_path, s_factor,**kwargs):
    """
    Scales the images size with a s_factor and apply grayscale
    """    
    gray = kwargs.pop('gray', False)
    image = open(img_path, "rb")
    if gray is True:
        img = PIL.Image.open(image).convert('L')
    else:
        img = PIL.Image.open(image)
    img = img.resize([int(s_factor * s) for s in img.size],
                     PIL.Image.ANTIALIAS)
    img = img.convert('RGB')
    img.save(img_path) 
    image.close()